In [14]:
# forbidden city
from PIL import Image, ImageEnhance, ImageFilter

# brightness adjust
img = Image.open('../Pic/city.png')
img_enhancer = ImageEnhance.Brightness(img)
enhanced_city = img_enhancer.enhance(0.75)

# sharpening
sharpened_city = enhanced_city.filter(ImageFilter.SHARPEN)

# compositing
img2 = Image.open('../Pic/sharpen_sky.png').convert("RGBA")
img2 = img2.resize((img.width, img.height), Image.LANCZOS)
result = Image.alpha_composite(img2, sharpened_city)

result.save('../Processed Pic/modified_city.png')
result.show()

In [15]:
# dragon
from PIL import Image, ImageFilter

# sharpening
img = Image.open('../Pic/dragon.png')
sharpened_dragon = img.filter(ImageFilter.SHARPEN)

# contrast adjust
enhancer = ImageEnhance.Contrast(sharpened_dragon)
result = enhancer.enhance(2)

result.save('../Processed Pic/modified_dragon.png')
result.show()

In [16]:
# moon
from PIL import Image
import numpy as np

def barrel_distort(image, k1, k2, k3):
    width, height = image.size
    x_center = width / 2
    y_center = height / 2
    
    # Create the coordinate grid
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x - x_center
    y = y - y_center
    
    r = np.sqrt(x**2 + y**2)
    
    # Calculate the distortion factor
    theta = np.arctan2(y, x)
    r_distorted = r * (1 + k1 * r**2 + k2 * r**4 + k3 * r**6)
    
    # Map the distorted coordinates back to image coordinates
    x_distorted = r_distorted * np.cos(theta) + x_center
    y_distorted = r_distorted * np.sin(theta) + y_center
    
    # Use bilinear interpolation to create the distorted image
    distorted_image = np.array(image)
    new_image = np.zeros_like(distorted_image)
    
    for i in range(height):
        for j in range(width):
            x_new = int(np.clip(x_distorted[i, j], 0, width - 1))
            y_new = int(np.clip(y_distorted[i, j], 0, height - 1))
            new_image[i, j] = distorted_image[y_new, x_new]
    
    return Image.fromarray(new_image)

def chromatic_aberration(image, shift_red, shift_blue):
    r, g, b = image.split()

    # Convert channels to numpy arrays
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)

    # Create shifted channels
    def shift_channel(channel, shift_x, shift_y):
        shifted = np.zeros_like(channel)
        if shift_y > 0:
            shifted[shift_y:] = channel[:-shift_y]
        elif shift_y < 0:
            shifted[:shift_y] = channel[-shift_y:]
        else:
            shifted[:] = channel[:]
            
        if shift_x > 0:
            shifted[:, shift_x:] = shifted[:, :-shift_x]
        elif shift_x < 0:
            shifted[:, :shift_x] = shifted[:, -shift_x:]
        
        return shifted
    
    r_shifted = shift_channel(r, *shift_red)
    b_shifted = shift_channel(b, *shift_blue)

    # Combine the channels back into an image
    result = np.stack([r_shifted, g, b_shifted], axis=-1)
    result_image = Image.fromarray(result)
    
    return result_image

# Load the image
img = Image.open('../Pic/moon.png').convert('RGB')

# barrel distortion
result_barrel_distort = barrel_distort(img, 0.000000000001, 0.000000000001, 0.000000000001)

# chromatic aberration distortion
result = chromatic_aberration(result_barrel_distort, (-5, -5), (5, 5))

result.save('../Processed Pic/modified_moon.png')
result.show()

In [17]:
# making reflection for objects
from PIL import Image, ImageFilter

# dragon reflection
img = Image.open('../Processed Pic/modified_dragon.png')
img = img.filter(ImageFilter.GaussianBlur)
img = img.transpose(Image.FLIP_TOP_BOTTOM)
img = img.resize((422, 100))

# opacity adjust
r, g, b, a = img.split()
a = a.point(lambda p: p * 0.25)
img = Image.merge("RGBA", (r, g, b, a))

img.save('../Processed Pic/reflection_dragon.png')
img.show()

In [18]:
# overlay
from PIL import Image 
  
img = Image.open('../Processed Pic/modified_city.png')  
img2 = Image.open('../Processed Pic/modified_dragon.png') 
img3 = Image.open('../Processed Pic/reflection_dragon.png')
img4 = Image.open('../Pic/shining_light.jpg')
img5 = Image.open('../Pic/boat_people.png')
img6 = Image.open('../Processed Pic/modified_moon2.png')
  
# dragon
img.paste(img2, (1000, 50), mask = img2) 

# dragon reflection
img.paste(img3, (1000, 1000), mask = img3)

# people on boats
img.paste(img5, (50, 650), mask = img5)

# moon
img.paste(img6, (0, 0), mask = img6)

# shining light effect on dragon
if img4.mode != 'L':
    img4 = img4.convert('L')

if img4.size != img.size:
    img4 = img4.resize(img.size)

img4 = img4.convert('RGB')
img = img.convert('RGB')

result_image = Image.blend(img, img4.convert('RGB'), 0.4)

result_image.save('../Processed Pic/result_v1.jpg')
result_image.show()

In [19]:
from PIL import Image
from moviepy.editor import ImageSequenceClip, AudioFileClip
import numpy as np

# Open base image
img = Image.open('../Processed Pic/result_v1.jpg').convert('RGBA')

# Open GIF
gif = Image.open('../Pic/shiny.gif')
gif2 = Image.open('../Pic/shiny2.gif')

# Prepare a list to hold frames
frames = []

# Get the number of frames (use the longer of the two GIFs)
n_frames = max(gif.n_frames, gif2.n_frames)

# Loop through each frame
for i in range(n_frames):
    # Create a new image with the same size as the base image
    new_frame = img.copy()

    # Process first GIF
    if i < gif.n_frames:
        frame1 = gif.seek(i)
        frame1 = Image.fromarray(np.array(gif.convert('RGBA')))
        new_frame.paste(frame1, (850, 0), mask=frame1)

    # Process second GIF
    if i < gif2.n_frames:
        frame2 = gif2.seek(i)
        frame2 = Image.fromarray(np.array(gif2.convert('RGBA')))
        new_frame.paste(frame2, (1110, 220), mask=frame2)

    # Convert PIL Image to numpy array and append to frames list
    frames.append(np.array(new_frame.convert('RGB')))

# Create a video clip
clip = ImageSequenceClip(frames, fps=24)

# adding sound effect for video
audio = AudioFileClip("../Pic/beam_charge.mp3")
video = clip.set_audio(audio)

# looping video to ensure video same duration as audio
video = video.loop(duration=audio.duration)

# Write the result to a file
video.write_videofile("../Processed Pic/result_v1.mp4", codec="libx264", audio_codec="aac")

Moviepy - Building video ../Processed Pic/result_v1.mp4.
MoviePy - Writing audio in result_v1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video ../Processed Pic/result_v1.mp4



Moviepy - Done !
Moviepy - video ready ../Processed Pic/result_v1.mp4
